In [3]:
from datasets import load_dataset, load_from_disk
from datasets import Dataset, DatasetDict
import numpy as np


ds_names = ["medium", "bigcodebench", "math-hard"]
ds_name = ds_names[1] # select one of the datasets


def load_sft_dataset(ds_name: str, 
                     add_system_prompt: bool=True
                     ) -> DatasetDict:
    '''
    Loads the dataset corresponding to Tab 5 in the paper,
    and transforms it into the SFT dataset where the conversations flow 
    with selecting the chosen response.
    '''
    dataset = load_from_disk(f"../data/interactive-{ds_name}")
    
    if add_system_prompt:
        dataset = add_sys_prompt(dataset, key='prompt')

    dataset_dict = {}
    for split in dataset.keys():
        prompts = dataset[split]['prompt']
        last_responses = dataset[split]['chosen']
        indices = dataset[split]['idx']

        sft_lst = [
            prompts[max(subset, key=lambda j: len(prompts[j]))] + 
            [{'role': 'assistant', 'content': last_responses[max(subset, key=lambda j: len(prompts[j]))]}]
            for subset in (np.where(indices == i)[0] for i in np.unique(indices))
        ]
        
        idx_lst = list(np.unique(indices))
        
        dataset_dict[split] = Dataset.from_dict({'chat': sft_lst, 'idx': idx_lst})
    
    return DatasetDict(dataset_dict)

In [6]:
sft_ds = load_sft_dataset(ds_name, add_system_prompt=False)['train']
print(sft_ds)

num_total_turns = sum(len(chat['chat']) for chat in sft_ds)
print(f"Total number of (user + assistant) turns: {num_total_turns}")

Dataset({
    features: ['chat', 'idx'],
    num_rows: 500
})
Total number of (user + assistant) turns: 5254


In [55]:
import random

rand_indices = random.sample(range(len(sft_ds)), 3)
rand_indices

[105, 334, 158]

In [61]:
from rich.console import Console
from rich.panel import Panel
from rich.json import JSON
from rich import box

console = Console()

for idx in rand_indices:
    chat = sft_ds['chat'][idx]
    
    # Create title with styling
    console.print(f"[bold cyan]Index: {idx}[/bold cyan]", justify="center")
    
    # Process each message in the chat
    for i, message in enumerate(chat):
        role_color = "green" if message["role"] == "assistant" else "yellow"
        console.print(Panel(
            message["content"],
            title=f"[{role_color}]{message['role'].capitalize()}[/{role_color}]",
            border_style=role_color,
            box=box.ROUNDED,
            expand=False
        ))
    
    # Add separator between different chats
    if idx != rand_indices[-1]:
        console.print("[dim]" + "─" * 80 + "[/dim]")
        console.print()

Index: 105

╭───────────────────────────────────────────────────── User ──────────────────────────────────────────────────────╮
│ Can you help me write the function 'task_func' that takes a DataFrame and draws a pie chart? I want it to raise │
│ a ValueError if the input is not a DataFrame. Make sure it returns the figure object, please.                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ Sure, I can help with that! To create the 'task_func' function, we'll use the Matplotlib library, which is      │
│ commonly used for plotting in Python. Here's a basic implementation:                                            │
│                                                                                                                 │
│ ```python                                                                                                       │
│ import pandas as pd                                                                                             │
│ import matplotlib.pyplot as plt                                                                                 │
│                                                                                                                 │
│ def task_func(df):                                                                                              │
│     if not isinstance(df, pd.DataFrame):                                                                        │
│         raise ValueError("Input must be a pandas DataFrame")                                                    │
│                                                                                                                 │
│     # Assuming we want to plot the first column as the data and the index as labels                             │
│     data = df.iloc[:, 0]                                                                                        │
│     labels = df.index                                                                                           │
│                                                                                                                 │
│     fig, ax = plt.subplots()                                                                                    │
│     ax.pie(data, labels=labels, autopct='%1.1f%%')                                                              │
│     ax.set_title('Pie Chart')                                                                                   │
│     plt.show()                                                                                                  │
│     return fig                                                                                                  │
│ ```                                                                                                             │
│                                                                                                                 │
│ This function checks if the input is a DataFrame, raises a ValueError if not, and then plots a pie chart using  │
│ the first column of the DataFrame as the data and the index as labels. It returns the figure object.            │
│                                                                                                                 │
│ Could you let me know if you have a specific column in mind for the pie chart, or if you'd like to use a        │
│ different plotting library? Also, feel free to ask if you need any further customization or explanation on how  │
│ this works!                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── User ──────────────────────────────────────────────────────╮
│ The function looks good, but can you explain how I can choose which column to use for the pie chart? I was      │
│ thinking I might want something other than the first column. Also, am I correct that the index is used for      │
│ labels? Thanks!                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ To allow for selecting a specific column for the pie chart, you can modify the function to accept an additional │
│ parameter for the column name. Here's how you can do it:                                                        │
│                                                                                                                 │
│ ```python                                                                                                       │
│ import pandas as pd                                                                                             │
│ import matplotlib.pyplot as plt                                                                                 │
│                                                                                                                 │
│ def task_func(df, column_name):                                                                                 │
│     if not isinstance(df, pd.DataFrame):                                                                        │
│         raise ValueError("Input must be a pandas DataFrame")                                                    │
│     if column_name not in df.columns:                                                                           │
│         raise ValueError(f"Column '{column_name}' not found in DataFrame")                                      │
│                                                                                                                 │
│     # Use the specified column for data                                                                         │
│     data = df                                                                                                   │
│     labels = df.index  # Confirming that the index is used for labels                                           │
│                                                                                                                 │
│     fig, ax = plt.subplots()                                                                                    │
│     ax.pie(data, labels=labels, autopct='%1.1f%%')                                                              │
│     ax.set_title('Pie Chart')                                                                                   │
│     plt.show()                                                                                                  │
│     return fig                                                                                                  │
│ ```                                                                                                             │
│                                                                                                                 │
│ With this change, you can call `task_func` and specify which column to use, like this:                          │
│                                                                                                                 │
│ ```python                                                                                                       │
│ task_func(your_dataframe, 'your_column_name')                                                                   │
│ ```                                                                                                             │
│                                                                                                                 │
│ This way, you have the flexibility to choose any column for the pie chart. Does this solution meet your needs   │
│ for selecting a specific column? If you have any other questions or need further customization, feel free to    │
│ ask!                                                                                                            │
╰───────────────────────────────────────────────────────

...

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ You're welcome! I'm glad to hear that the function seems to meet your needs. Good luck with your testing, and   │
│ feel free to reach out if you have any questions or need further assistance. Enjoy working with your data!      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── User ──────────────────────────────────────────────────────╮
│ Great! I think I'm all set with this function as `task_func(data_dict, data_keys)`. I’ll just check the input   │
│ data format when I run my tests. Thanks for your help!                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ Great to hear that you're ready to test the function! As you proceed, make sure to validate the correlation     │
│ coefficient against known values or datasets to ensure accuracy. Also, double-check that your input data is in  │
│ the expected format, with numeric values that can be processed by pandas. If you run into any specific issues   │
│ or need further assistance, don't hesitate to reach out. Happy testing!                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── User ──────────────────────────────────────────────────────╮
│ Thanks for your help! I feel ready to test the function now and will check the input data format. If I have     │
│ questions later, I'll reach out. Cheers!                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Assistant ───────────────────────────────────────────────────╮
│ It's great to hear that you're ready to test the function! As you prepare your data, here are a few tips to     │
│ ensure everything goes smoothly:                                                                                │
│                                                                                                                 │
│ 1. **Data Type Consistency**: Make sure your data series contain numeric values since the correlation           │
│ calculation requires numerical input.                                                                           │
│ 2. **Handling Missing Values**: Check for any missing values in your data. You can handle these by filling them │
│ with a default value or dropping them, depending on your analysis needs.                                        │
│ 3. **Data Range and Scaling**: Ensure that both data series are on a comparable scale if needed, as extreme     │
│ differences might affect the correlation.                                                                       │
│                                                                                                                 │
│ If you encounter any issues or have questions about the results, feel free to reach out. I'm here to help you   │
│ refine the process and ensure your testing is successful. Happy testing!                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯